# Sparse Subspace Clustering (Vidal IEEE Trans. on PAMI 2013) on Yale Dataset

#### References:


1.   Main Paper (Vidal IEEE Trans. on PAMI 2013): https://arxiv.org/pdf/1203.1005.pdf
2.   Supplementary Paper (Vidal CVPR 2009): http://cis.jhu.edu/~ehsan/Downloads/SSC-CVPR09-Ehsan.pdf
3.   Spectral Clustering: http://people.csail.mit.edu/dsontag/courses/ml14/notes/Luxburg07_tutorial_spectral_clustering.pdf
4.   Spectral Clustering Code: https://juanitorduz.github.io/spectral_clustering/
5.   Yale Dataset: http://vision.ucsd.edu/content/yale-face-database

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Our objective is given a set of points drawn from a `union of subspaces`, we need to find the number of subspaces, their dimensions, a basis for each subspace, and the segmentation of the data.

In [106]:
import numpy as np
import cvxpy as cp
from scipy.sparse import identity
from cvxpy.atoms.elementwise.power import power

##### Set Parameters of the data space

In [107]:
N = 165 # Total no of points
D = 1024  # Dimension of space
K = 15 # Number of clusters

In [108]:
yale_data = np.load('yale_X.npy')
input_data = yale_data.astype('float32') / 255.
input_data = input_data.T
orig_label = np.load('yale_Y.npy')

The matrix *input_data* contains the data points arranged sequentially forming a matrix of shape $(1024, 165), ie. (D,N)$. The *input_data* matrix is of the form $[y[0],y[1], \ldots, y[N-1]], $where each $y_i$ denotes the vector of a data point of dimension $D$.

Also, *input_data* can be assumed to be of the form $[Y_0, Y_1, \ldots, Y_n]$, where each $Y_i$ denotes the set of $N_i$ data points belonging to subspace $i$. Also assume that the dimension of each subspace $i$ is $d_i$ and $A_i$ is its basis.

In [109]:
def zij(Y,i,j,lam,N):
    if i==j:
        return 0.0
    else:
        numerator = 2 * np.exp(-(np.linalg.norm(Y[:,i]-Y[:,j]))/lam)
        print(numerator)
        sum_i=0
        sum_j=0
        for h in range(N):
            if h!=i:
                sum_i += np.exp(-(np.linalg.norm(Y[:,i]-Y[:,h]))/lam)
        for h in range(N):
            if h!=j:
                sum_j += np.exp(-(np.linalg.norm(Y[:,j]-Y[:,h]))/lam)
        return numerator/(sum_i+sum_j)

In [110]:
Z =np.zeros((N,N), dtype='float64')
for i in range(N):
    for j in range(N):
        Z[i,j] = zij(input_data,i,j,199101,N)

1.9998628921880877
1.999930982977655
1.9999215980945673
1.9999129126895667
1.9999128545618459
1.9998909180201128
1.9998449798465259
1.9998388022999198
1.999906631150357
1.9999055905163219
1.9998943572151722
1.9999206026772047
1.9999226830657626
1.9999110559465372
1.9998414137182423
1.9998897925314412
1.999904331914374
1.999934345083611
1.999917060798451
1.9999104412091846
1.9999149334814859
1.9998996235467552
1.9999327981929693
1.9998912482275661
1.9998406874064625
1.9999138971346981
1.999928311704852
1.9999354595569871
1.9999062667803822
1.9998861075404344
1.999857700555436
1.99988912182893
1.9999230478698031
1.9999240255376796
1.999902162528926
1.9998547371602071
1.9998948360460376
1.9998874756985479
1.9998979031967712
1.9998905501565634
1.9998507946520545
1.9999244663500508
1.9998952760904203
1.9998615206078807
1.999879153278819
1.99988497910406
1.9998983621130844
1.999871558136053
1.9998602887307169
1.9998647073224416
1.9999213759140968
1.99992479455136
1.9998902768788784
1.9998572

1.9998941955166223
1.999835208882459
1.999926279624401
1.9999132513113551
1.9998961841702116
1.9998911472331828
1.9998927824366166
1.999915131679218
1.9998995802577924
1.9998962183012832
1.9999196829856485
1.9999170911268542
1.9999524724728273
1.9999216815505532
1.9999260394867309
1.9998588165618685
1.999969001820583
1.9999139992893364
1.9999191477595943
1.9999202308231554
1.9999069110687722
1.999925744995297
1.999914470940359
1.9999119531896323
1.9999215980945673
1.9998798209796882
1.999926050689898
1.999926025409729
1.9999173820438945
1.999911247476637
1.9998522789517135
1.9998344540562154
1.9999039621899644
1.9999161949064501
1.9999079260499595
1.9999684363982373
1.9999251173020187
1.9999163715027515
1.9998512769688583
1.9999089978564748
1.9998956528252751
1.9999286653306863
1.9999069786032508
1.9999054312117015
1.9999167994435039
1.9998963849046023
1.999925827962904
1.9998725102597863
1.9998505479330353
1.9999263881595868
1.999921294991882
1.9999308485680627
1.9998868552304652
1.99

1.9998328824092686
1.9999080220256031
1.9999153210351848
1.9999080220256031
1.9998851604201775
1.9999274747762577
1.9999094722364381
1.9998444761220426
1.9999087445971466
1.9999011830229265
1.9998405352529764
1.9998732749957049
1.9998321325841413
1.999904222882259
1.9999055896350213
1.9998666631284867
1.9998914667885097
1.9999141974294998
1.9998617995141719
1.9999052822240546
1.9999094550797853
1.9998817696160853
1.9999273053539883
1.999902785837242
1.9998941906982355
1.9998969628051837
1.9998726879825754
1.9998992540722784
1.999904133133372
1.9998399478950324
1.9999116891722846
1.9999624402508231
1.9998866128655612
1.9998732559905774
1.9998994906049068
1.9999734645992728
1.9999073873257953
1.9999103168688956
1.9999660136958313
1.9999695505217943
1.9999103918658225
1.999926463827678
1.9999195031272625
1.9998677303819403
1.9999157695043683
1.9999016271590813
1.999926134155649
1.9999112254344855
1.99990601455597
1.9999183681772026
1.9999052967559297
1.9998938691691386
1.9998909180201128


1.9998416196389783
1.9998673091954227
1.9998502242652392
1.9998375445925372
1.9998226629984288
1.9998283107965629
1.9998286888598606
1.999815090427398
1.999820181686634
1.999832816773935
1.9998334564407392
1.9998435310618405
1.999845712347941
1.9998465342414968
1.9998206351538503
1.9998133114888843
1.9998409081650603
1.9998405429736832
1.9998937016751546
1.9998521948855614
1.9998441053262652
1.9998521948855614
1.9998413829407717
1.9998398778914257
1.9998338293706612
1.9998689423868963
1.9998281535863165
1.999850846331244
1.9999453535236342
1.9998494169974266
1.9998675355320361
1.9998226629984288
1.9998414318609532
1.9998478991156516
1.9998435990060783
1.9998474799065997
1.9999297234544673
1.9998231499542425
1.9998490922186403
1.9998407336157582
1.9998337700382838
1.9998511057809123
1.9998294638764338
1.9998409081650603
1.9998557654490414
1.9998432773510082
1.9998509262400141
1.9998694592601758
1.9998294227059459
1.9998366774034797
1.9998425647365934
1.999862347096344
1.9998473287200833

1.9998597169522196
1.9998702436130942
1.9999075012052394
1.9998743031312245
1.9998994812363289
1.9998856258666196
1.9999364104466495
1.99990462544472
1.9999023633022341
1.9998690490127142
1.999839829134126
1.9999156345880902
1.9999140208908666
1.99989406713488
1.999870948938032
1.9998964369298253
1.9999035714772835
1.9998812624529692
1.9999071756794096
1.9998497015728478
1.9998662900104378
1.999853067234904
1.999918417233379
1.9999422275872023
1.9998606713309917
1.999910061032549
1.999894504659738
1.999875978390722
1.9998815933389542
1.9998987436573388
1.9998401267835704
1.999889604442356
1.9999009923750888
1.9999082640003478
1.9999092317655083
1.9998841156782372
1.9999177431972126
1.999839403777025
1.999870948938032
1.9999748794366807
1.9998974705853576
1.9998571702361108
1.99992024417687
1.999909607784539
1.99992024417687
1.9999196827509527
1.9999127992408867
1.999880539307022
1.9998675197264058
1.9998864918220194
1.9999071731121412
1.999846537699541
1.9999054017264581
1.999851002163

...

1.9999125400801758
1.9999310837633015
1.999974897108803
1.9998466677545692
1.999902281829977
1.9998901191369003
1.9999269424267443
1.9998946630342957
1.9998912216354754
1.999959318394942
1.9998939728075449
1.999924188699506
1.9998657188049742
1.9998460975892496
1.9999103569585797
1.9999166166301308
1.999928843925407
1.9998690588026333
1.999895270256625
1.9998706176227448
1.9998607389884377
1.9999199000505203
1.9999061120546395
1.9999226734432212
1.9998679654931228
1.9999166006804214
1.9999062693476493
1.999920885088226
1.9999074977949869
1.999842833419393
1.9999003145130645
1.9999189145678573
1.999874126548217
1.999850422724773
1.999855680127879
1.9998710057905031
1.99985166251777
1.9998728238442967
1.9998856944540977
1.9999066338229978
1.9998999987784747
1.9999135690309708
1.9998693933554246
1.9998938932035923
1.9999142758943127
1.9998697432828565
1.9999043138285686
1.999906918301188
1.9998557777198858
1.9998979626268445
1.9998636182109784
1.9999152142632088
1.9999266496834276
1.99990

1.9999324881722014
1.9999268827323748
1.9998932756073067
1.9998734793008373
1.99989761262742
1.9999264237711991
1.9998978695446368
1.9999021315972512
1.999927615321266
1.999926906877389
1.9999149386639397
1.999931558546052
1.9998715367744613
1.999917124961321
1.999905479022245
1.9999718177983428
1.9999326205750025
1.9999010019735777
1.9999282882351728
1.9999046032110546
1.9998989027315228
1.999922913714599
1.9998810299553544
1.9999260394867309
1.9999682894990507
1.9999292642627327
1.9999195031272625
1.9999130677798522
1.9998542187580757
1.9998358349669807
1.9999055811572948
1.9999180939090369
1.9999088184061729
1.9999650985888027
1.9999286162598893
1.999919304172323
1.9998528360433245
1.9999104948824258
1.9998969259344166
1.9999311563038717
1.9999074920377908
1.999904097603581
1.9999191244720316
1.9998971009388509
1.9999277847387715
1.9998727640701193
1.9998522624469297
1.9999276562447095
1.9999214000494652
1.9999329924696734
1.9998857532324186
1.9999011043957786
1.9998744682096248
1.9

1.9998978163315415
1.9999075899197067
1.9998978163315415
1.9998902222674106
1.9999161355045731
1.9999560880861622
1.9998374213104733
1.9999252204630722
1.9999078711027318
1.9998472794070155
1.9998613041767788
1.9998247991639992
1.9999250067504286
1.9999308391227022
1.9998540444183521
1.9999067651559888
1.9998962347681126
1.9998572210437653
1.9999257466381724
1.9998953949695712
1.999904659058651
1.9999181477453334
1.9999108240010626
1.9998942418900023
1.9998866624765559
1.9998831591521053
1.9998868920148334
1.9998896270590258
1.9998328941531276
1.9999250331657488
1.9999118575969752
1.9998928946771704
1.9998935265943834
1.9998896735089342
1.9999134151476496
1.9998966859152227
1.9998925079899532
1.999917851751094
1.9999148548825356
1.9999618363058842
1.999917124961321
1.9999211184382533
1.9998560691452794
1.999908399337621
1.9999183954018955
1.9999173089051108
1.9999025494862313
1.9999218063992585
1.9999093495055438
1.9999122822606121
1.9999067245969884
1.9998762835652282
1.99991399928933

1.999847986084083
1.9999069953862874
1.9999043423654406
1.999872746607247
1.9998997385273796
1.9999256624683481
1.9999822953676585
1.9999071756794096
1.999903214809656
1.9998337798471881
1.9998396433487085
1.9998875893852728
1.9999282512153465
1.999895975389224
1.9998432811634717
1.9998738115745185
1.9999485750641495
1.9999054827198748
1.999913586599573
1.9999163594518616
1.99988101711918
1.9999005974959805
1.999908472428182
1.999932225818972
1.9999155634418855
1.9999090729874633
1.9999026320409645
1.9998735383661752
1.9998997385273796
1.9999044799152774
1.9999114966842042
1.9998334091779209
1.9999059845151155
1.9999140284202692
1.9999059845151155
1.9999010485099642
1.9999221069538924
1.9999092898069097
1.999849586585521
1.9999212429564333
1.9999272623757587
1.9998536083966858
1.9998705066862104
1.99983457061363
1.9999054827198748
1.9999259462068375
1.9998643929728717
1.9999096566489134
1.9999048932641368
1.999864485775726
1.999905766431553
1.9999001693192118
1.9998983044934902
1.99993

1.999915453412901
1.9998684281192858
1.9999230856798365
1.9999072724501041
1.9998708587978915
1.9999752346992956
1.9999382655128557
1.999858642978495
1.9998846518863684
1.999870046502258
1.9999126591710994
1.9999048544390363
1.9999667860593737
1.9999154483358192
1.9999238636068244
1.99991590354893
1.9999002284619727
1.9999465372536984
1.9999098394233328
1.9998414173391212
1.999890530212502
1.9999036782199
1.9998877823595946
1.9999080318540305
1.9999090316620665
1.9999256218706465
1.9999218444918616
1.9999072194570842
1.9998879224084813
1.9998509015738566
1.9999259591869918
1.9999216297066829
1.999881131332396
1.9998692161597833
1.999928520786179
1.9999230430226087
1.9999123421796614
1.9999040817593443
1.9998325014523195
1.9998467821190884
1.9998613948337622
1.9999275756958397
1.9998958324851404
1.9998417795707328
1.9998812351904647
1.9999070650952835
1.999888746829154
1.9998983904774439
1.9999042947752492
1.9998583252388678
1.9998878825778872
1.999898222494536
1.9999038052994684
1.9999

In [111]:
D = np.zeros((N,N))
sum_list=[]
for i in range(N):
    csum = np.sum(Z[:,i],axis=0)
    sum_list.append(csum)

D = np.diag(sum_list)
print(D)

[[1.00000304 0.         0.         ... 0.         0.         0.        ]
 [0.         0.99999758 0.         ... 0.         0.         0.        ]
 [0.         0.         1.0000033  ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.0000043  0.         0.        ]
 [0.         0.         0.         ... 0.         1.00000316 0.        ]
 [0.         0.         0.         ... 0.         0.         1.00000064]]


$D$ is the `degree matrix`. In this case, $D \in \mathbb{R}^{NxN}$ is a diagonal matrix with $D_{ii} = \sum_{j}W_{ij}$. 

In [112]:
L = np.subtract(D, Z)
LN =  np.diag(np.divide(1, np.sqrt(np.sum(D, axis=0) + np.finfo(float).eps)))@ L @  np.diag(np.divide(1, np.sqrt(np.sum(D, axis=0) + np.finfo(float).eps)))
print(LN.shape)

(165, 165)


This $L$ is the Laplacian matrix, which can be defined as $L = D - W$. Next, we calculate the `eigenvalues` and `eigenvectors` of the Normalized Laplacian matrix, which we will use for Spectral clustering of the data points. $L$ is a *positive, semi-definite matrix* this means all the eigenvalues of the matrix will be greater than equal to $0$.

### Perform Spectral Clustering with Laplacian Matrix LN

In [113]:
from scipy import linalg

eigenvals, eigenvcts = linalg.eig(LN)

eigenvals = np.real(eigenvals)
eigenvcts = np.real(eigenvcts)

eig = eigenvals.reshape((N,1))

Sort Eigen Values

In [114]:
eigenvals_sorted_indices = np.argsort(eigenvals)
eigenvals_sorted = eigenvals[eigenvals_sorted_indices]

In [115]:
indices = []
for i in range(0,K):
    ind = []
    print(eigenvals_sorted_indices[i])
    ind.append(eigenvals_sorted_indices[i])
    indices.append(np.asarray(ind))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In the above code, we find out the indices of the eigenvectors corresponding to the $K$ smallest eigenvectors.

In [116]:
indices = np.asarray(indices)

In [117]:
zero_eigenvals_index = np.array(indices)

Here, the indices are arranged according to their sorted order of values and the sorted eigen values are stored in *eigenvals_sorted*.

In [118]:
eigenvals[zero_eigenvals_index]

array([[2.63472335e-17],
       [1.00608924e+00],
       [1.00609262e+00],
       [1.00609451e+00],
       [1.00609521e+00],
       [1.00609575e+00],
       [1.00609610e+00],
       [1.00609634e+00],
       [1.00609655e+00],
       [1.00609659e+00],
       [1.00609674e+00],
       [1.00609690e+00],
       [1.00609696e+00],
       [1.00609718e+00],
       [1.00609720e+00]])

In [119]:
import pandas as pd

proj_df = pd.DataFrame(eigenvcts[:, zero_eigenvals_index.squeeze()])
proj_df.columns = ['v_' + str(c) for c in proj_df.columns]
proj_df.head()

,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0.07785,0.077772,0.021564,0.027693,-0.007208,0.006871,-0.041433,-0.088548,-0.020452,-0.071359,0.048208,0.041358,-0.087182,0.073867,-0.131947
1,0.07785,-0.128399,0.012116,0.046921,0.147035,0.063363,0.128517,0.014660,0.087984,-0.178041,-0.023945,0.046839,0.032113,-0.015505,-0.016208
2,0.07785,0.073276,-0.005316,0.034264,-0.091886,0.042797,0.065413,0.053111,0.057993,-0.048009,0.078218,-0.002624,-0.231163,-0.051964,0.047338
3,0.07785,0.038496,-0.046541,0.075860,-0.021089,0.087963,-0.005964,-0.087549,0.066136,0.071045,-0.227184,-0.061105,-0.067951,0.004818,0.008372
4,0.07785,0.022744,-0.072770,0.001014,-0.039595,-0.049832,0.014933,-0.000819,0.082916,0.043896,-0.042332,0.025724,-0.027001,-0.006480,-0.160288


Stack the Eigen Vectors corresponding to the zero Eigen Values in a dataframe *proj_df*. This can be thought of as a $N X K$ matrix where the columns denote an eigen vector and the rows denote a data point.

Apply *K-Means Clustering* with $K = 15$.

In [120]:
from sklearn.cluster import KMeans

def run_k_means(df, n_clusters):
    k_means = KMeans(random_state=25, n_clusters=n_clusters)
    k_means.fit(df)
    cluster = k_means.predict(df)
    return cluster

cluster = run_k_means(proj_df, n_clusters=K) +1

*run_k_means* applies `K-Means Clustering` on *proj_df* with number of clusters = $3$.The clustering of the data points is returned in the variable *'cluster'*.

Display clusters formed

In [121]:
print(cluster)

[ 6  8  5  7  6 12  3  4  4  9 13 15  7  6 12 10  3  9 11  2  9 12  4 11
  1 10  6  5 11  1 15  8  1  7  5  6  8  3  3  3  3  4  2  3  8  1  1  1
  1  8  4  5  2  3  8 14  6  1 13 14  1  2  1  7 11 13  5 14  6 15  6 14
 10  4 11  9  6  5 14  6  7  9  4 14  3  4  1  6  6  6  6 10  8  1  6 15
 10  8  6  2  2  7  1  2  2 11  6  9  3  1  1 15 12  4  3  6  3 14  6  5
 10  2 13  4  8 10  2  6  8  9  3  4  2  3  9 11 13  1 15  1 15  3 10  2
 12 14  9  3 12  3  3 12 12  5 11  7  8  5 13 11  6  3  7 13  9]


As we can see, the data points have been clustered into $fifteen$ subspaces: $1$ to $15$ corresponding to the $fifteen$ subspaces that we have considered.

In [122]:
pred = np.asarray(cluster)

### Calculate Performance

In [123]:
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score
print("ARI = " + str(adjusted_rand_score(orig_label, pred)))
print("NMI = " + str(normalized_mutual_info_score(orig_label, pred)))

ARI = 0.3975251353441609
NMI = 0.6477782292351937


In the above code block, we calculate the `Adjusted random score` and the `normalized mutual info score` between the `original` and the `predicted` labels for the various data points.